## Exploratory Data Analysis

### Introduction

In this project, I will be developing and deploying a web application to a cloud service to be viewed by the public.

I have a used cars inventory dataset and will be performing exploratory data analysis, cleaning the data, then develop the web application with Streamlit and deploy it to Render.

I hope to practice my software development skills, my project structure skills, and my data analytic skills with this task.


In [76]:
import pandas as pd
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

In [77]:
#read data into dataframe
file_path = r'..\vehicles_us.csv'
df_raw = pd.read_csv(file_path)

In [78]:
#check the data for things we would want to fix
display(df_raw.sample(10))
df_raw.info()

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
7025,11250,2010.0,chevrolet silverado 1500,good,8.0,gas,121334.0,automatic,pickup,grey,1.0,2018-08-09,43
26115,26900,2016.0,chevrolet colorado,excellent,6.0,gas,29676.0,automatic,truck,grey,1.0,2019-03-04,34
33076,29995,2016.0,gmc acadia,good,6.0,gas,30613.0,automatic,SUV,white,1.0,2018-07-29,11
32154,29999,NaN,gmc sierra 2500hd,excellent,8.0,gas,59885.0,automatic,pickup,NaN,1.0,2018-10-14,104
25092,11995,2012.0,ford explorer,excellent,6.0,gas,121467.0,automatic,wagon,black,1.0,2019-03-20,63
3815,5900,2006.0,honda accord,excellent,6.0,gas,113788.0,automatic,sedan,NaN,NaN,2019-01-30,8
46642,17995,2013.0,chevrolet silverado 1500,excellent,8.0,gas,91100.0,automatic,truck,black,1.0,2018-12-08,30
2470,21000,2011.0,toyota tundra,good,8.0,gas,155907.0,automatic,truck,black,1.0,2019-04-11,9
12814,22995,2015.0,ram 3500,excellent,6.0,diesel,142469.0,automatic,other,silver,NaN,2018-09-01,19
12231,1,2014.0,toyota camry,like new,4.0,gas,NaN,automatic,sedan,red,NaN,2018-11-17,18


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


##### Missing Values
###### - There are a good amount of NaN model years, I don't love the idea of filling them with 0, but I would like to convert the column to int and I can always exclude 0's from visualizations later on.
###### - There are a lot of NaN cylinder values as well. We could replace this with the most common cylinder based on the type of car. I will remove this if needed.
###### - There are lots of missing odometer values. i will replace these with 0.
###### - There are missing paint_colors. I can fill these with 'black', but I think it's fine to leave them as NaN
###### - There are 10s of thousands of missing 4wd values. If we check below the unique values of this field, it's either 1 or NaN. I will fill these missing values with 0

In [79]:
#The model field should be split up into two different columns so we can create visualizations or drop downs based on the brand
#split the column up into 5 columns by spaces
df_raw[['make','model','filler','filler1','filler2']] = df_raw['model'].str.split(' ',expand=True)

In [80]:
#combine the last 4 columns back into the model column
df_raw['model'] = df_raw['model'] + ' ' + df_raw['filler'].fillna('') + ' ' + df_raw['filler1'].fillna('') + ' ' + df_raw['filler2'].fillna('')

In [81]:
#drop the filler columns
df_raw.drop(['filler','filler1','filler2'],axis=1,inplace=True)

In [82]:
#add an age column
df_raw['age'] = 2024 - df_raw['model_year']

In [83]:
#check unique values of is_4wd
print(df_raw['is_4wd'].unique())

[ 1. nan]


In [84]:
#replace those missing values with 0
df_raw['is_4wd'] = df_raw['is_4wd'].fillna(0).astype(int)

In [85]:
#replace those missing values in odometer with 0
df_raw['odometer'] = df_raw['odometer'].fillna(0).astype(int)

In [86]:
#Fill missing model_year values and convert to int
df_raw['model_year'] = df_raw['model_year'].fillna(0).astype(int)

#### Edit

Filling the NaN's with the median cylinder by model/year instead of filling them by type of car.

In [87]:
#alternative method for filling the NaN cylinders. I will grab the median cylinder from the dataframe grouped by model and model_year and use that value
def cylinder_estimation(df):
    # Calculate median cylinders grouped by car model and model year
    median_cylinders = df.groupby(['model', 'model_year'])['cylinders'].transform('median')

    # Fill NaN values with the calculated median
    df['cylinders'] = df['cylinders'].fillna(median_cylinders)
    
    return df

df_raw = cylinder_estimation(df_raw)

In [88]:
#checking if the NaN's are gone
df_raw[df_raw['cylinders'].isna()].count()

price           26
model_year      26
model           26
condition       26
cylinders        0
fuel            26
odometer        26
transmission    26
type            26
paint_color     23
is_4wd          26
date_posted     26
days_listed     26
make            26
age             26
dtype: int64

After replacing the NaN cylinders with the median for each model and year, there are still 26 cars with NaN cylinders. This is probably because there are few models of those cars and none of them have a cylinder value. I plug the rest with my original function cyl using the averages by car type

In [89]:
#check the unique values of cylinder and type to see what we need to assign default values to.
print(df_raw['cylinders'].unique())
print(df_raw['type'].unique())

[ 6.   4.   8.   5.  10.   7.   nan  3.   4.5 12.   9.   5.5]
['SUV' 'pickup' 'sedan' 'truck' 'coupe' 'van' 'convertible' 'hatchback'
 'wagon' 'mini-van' 'other' 'offroad' 'bus']


In [90]:
#see the average cylinders for each type of car.
df_cyl_avg = df_raw[['type','cylinders']]
df_cyl_avg.groupby(['type']).mean().round()

#I will round convertibles down to 6,pickups + trucks to 8 since no 7 cylinder engines exist in the data, and buses to 10 since 9 doesn't exist.

,cylinders
type,
SUV,6.0
bus,9.0
convertible,7.0
coupe,6.0
hatchback,4.0
mini-van,6.0
offroad,6.0
other,6.0
pickup,7.0


In [91]:
#create a function for assigning default values of cylinders for each car type.
def cyl(x):
    if x in ['SUV','convertible','coupe','mini-van','offroad','van']:
        return 6
    elif x == 'bus':
        return 10
    elif x == 'hatchback':
        return 4
    elif x in ['pickup','truck']:
        return 8
    elif x == ['sedan','wagon']:
        return 5
    else:
        return 6 #this is for 'other' type

In [92]:
df_raw['cylinders'] = df_raw['cylinders'].fillna(df_raw['type'].apply(cyl)).astype(int)

##### Data Types

In [93]:
df_raw['price'] = df_raw['price'].astype(float)

In [94]:
#preview new dataframe
df_raw.sample(5)

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,make,age
15031,999.0,2013,explorer,like new,6,gas,139000,automatic,SUV,NaN,1,2019-01-14,7,ford,11.0
41088,37840.0,2016,yukon,excellent,8,gas,67273,automatic,SUV,black,1,2019-02-11,3,gmc,8.0
12804,7650.0,1997,f-250,good,8,diesel,314000,automatic,truck,NaN,1,2019-02-20,5,ford,27.0
2476,33311.0,0,wrangler,excellent,6,gas,28286,automatic,SUV,white,1,2019-01-02,56,jeep,NaN
19165,4900.0,2001,silverado,good,8,gas,189000,automatic,pickup,red,1,2018-08-17,31,chevrolet,23.0


In [95]:
#checking the dataframe to make sure data types and missing values look good and look at a sample
display(df_raw.sample(10))
df_raw.info()

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,make,age
9164,5800.0,2008,cr-v,good,4,gas,142000,automatic,SUV,red,0,2019-02-28,61,honda,16.0
35620,8500.0,2014,passat,excellent,4,diesel,113000,automatic,sedan,NaN,0,2018-12-26,27,volkswagen,10.0
13912,26900.0,0,silverado 1500 crew,good,8,gas,0,automatic,pickup,NaN,1,2018-09-27,57,chevrolet,NaN
1618,10994.0,2015,equinox,excellent,4,gas,117488,automatic,SUV,black,1,2018-07-20,11,chevrolet,9.0
46267,7498.0,2016,altima,good,4,gas,82068,automatic,sedan,grey,0,2018-07-11,21,nissan,8.0
36542,7995.0,2006,f-150,good,8,gas,149905,automatic,truck,silver,1,2018-06-30,15,ford,18.0
50686,14700.0,2012,f-150,excellent,6,gas,139000,automatic,pickup,NaN,1,2018-10-02,25,ford,12.0
10530,17500.0,2013,1500,excellent,8,gas,104140,automatic,truck,NaN,0,2018-10-31,36,ram,11.0
1432,16800.0,0,escape,excellent,4,gas,79717,automatic,SUV,blue,1,2018-08-14,80,ford,NaN
34575,5600.0,2000,f-250,excellent,8,diesel,0,automatic,pickup,white,0,2019-01-01,55,ford,24.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  float64
 1   model_year    51525 non-null  int32  
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     51525 non-null  int32  
 5   fuel          51525 non-null  object 
 6   odometer      51525 non-null  int32  
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        51525 non-null  int32  
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
 13  make          51525 non-null  object 
 14  age           47906 non-null  float64
dtypes: float64(2), int32(4), int64(1), object(8)
memory usage: 5.1+ MB


##### Duplicates

##### Student Comment

I will add date posted to the duplicate check to get a more accurate representation of duplicates. I believe that more duplicates (around 7500) should be removed, but that would be 13% of the data. From researching, I believe that there are a lot of duplicates in this dataset, however.

In [96]:
#Check the data for duplicates
dups = df_raw.duplicated(subset=['make','model','price','model_year','odometer','cylinders','fuel','transmission','type','is_4wd','paint_color','date_posted'])

dup_rows = df_raw[dups]

dup_value_counts = dup_rows.groupby(['make','model','price','model_year','odometer','cylinders','fuel','transmission','type','is_4wd','paint_color','date_posted']).size()

test = dup_value_counts.sum()

print(dup_value_counts.sample(10))
print(test)

make       model                 price    model_year  odometer  cylinders  fuel    transmission  type    is_4wd  paint_color  date_posted
ford       econoline             11990.0  2011        146100    8          gas     automatic     truck   0       white        2018-09-26     1
           f-350 sd              28990.0  2015        54362     8          diesel  automatic     truck   1       white        2018-09-08     1
kia        soul                  1865.0   2014        95412     4          gas     automatic     SUV     0       black        2019-03-15     1
chevrolet  silverado             23995.0  2015        115123    8          gas     automatic     pickup  1       red          2018-07-13     1
ford       f-150                 11500.0  2009        0         8          gas     automatic     pickup  0       green        2018-07-16     1
honda      civic                 10985.0  2013        52940     4          gas     automatic     sedan   0       black        2019-01-22     1
    

In [97]:
#check for duplicates
df_raw.duplicated(subset=['make','model','price','model_year','odometer','cylinders','fuel','transmission','type','is_4wd','paint_color','date_posted']).sum()

72

### I believe I need to get rid of rows that have dupilcate model, year, price and odometer reading and prioritize keeping those with less NaN values

In [ ]:
#sort the records by the records with less NaN values to be on top
df_sorted = df_raw.iloc[df_raw.isnull().sum(axis=1).argsort()]

In [ ]:
#drop duplicates based on the columns I determined and keep the first records based on the results from above
df = df_sorted.drop_duplicates(subset=['make','model','price','model_year','odometer','cylinders','fuel','transmission','type','is_4wd','paint_color','date_posted'],keep='first')

In [ ]:
#new info
df.info()

## Visualizations

#### Histograms

In [ ]:
#Price distribution
#There's not many cars above $50k, so I excluded them from the distribution.
fig1 = px.histogram(df,
                    x='price',
                    color='type',
                    range_x=[0,50000],
                    nbins=500,
                    opacity=.6,
                    title='<b> Price Distribution by Type of Car <b>',
                    template='plotly_dark')

fig1.update_layout(yaxis_title='Amount of Cars',xaxis_title='Price (USD)',height=800)
fig1.show()

##### We can tell from the above histogram that most of the cars are between the $2k to $10k range. The higher prices are mostly occupied by trucks and SUVs, but this data doesn't contain many high priced vehicles. Sedans don't appear to have many high priced cars in the data.

In [ ]:
#Brand distribution of modern luxury cars

#create a filtered dataframe of luxury models in between 2015 and 2020
filtered_df_year = df[(df['model_year'] >= 2010) & (df['model_year'] <= 2020) & (df['make'].isin(['bmw','acura','mercedes-benz','cadillac','buick','lexus','audi','lincoln']))]

#plot histogram of brands with a color filter of years
fig2 = px.histogram(filtered_df_year,
                    x='make',
                    color='model_year',
                    title='<b> Modern Luxury Car Model Year Distribution <b>',
                    template='plotly_dark')

fig2.update_layout(yaxis_title='Amount in Inventory',xaxis_title='Brand',height=800)

fig2.show()

#### Based on the above histogram, buick has the biggest stock of modern luxury cars and mercedes-benz only has 34 from one year. It appears that most of the luxury vehicles in the inventory come from 2011 and 2012.

In [ ]:
#Create year ranges as eras and plot the distribution of all cars in these ranges

#create year_range function to pass the dataframe to
def year_range(x):
    if 1920 <= x <= 1940:
        return '1920-1940'
    elif 1940 < x <= 1960:
        return '1941-1960'
    elif 1960 < x <= 1980:
        return '1961-1980'
    elif 1980 < x <= 2000:
        return '1981-2000'
    elif 2000 < x <= 2020:
        return '2001-2020'
    else:
        return 'unknown'

In [ ]:
#create year_range column in dataframe. I will add this to my functions and to the dataframe for the app
df['year_range'] = df['model_year'].apply(year_range)

In [ ]:
#create condition column based on odometer readings. These are opinion based
def condition(x):
    if 0 <= x <= 200:
        return 'new'
    elif 200 < x <= 5000:
        return 'like new'
    elif 5000 < x <= 20000:
        return 'good'
    elif 20000 < x <= 50000:
        return 'used'
    elif 50000 < x <= 100000:
        return 'very used'
    else:
        return 'heavily used'

In [ ]:
#add condition column
df['condition'] = df['odometer'].apply(condition)

In [ ]:
#plot histogram of 
fig3 = px.histogram(df,
                    x='make',
                    color='year_range',
                    title='<b> Distribution of Brands by Year Range <b>',
                    template='plotly_dark')

fig3.update_layout(yaxis_title='Amount in Inventory',xaxis_title='Brand',height=800)

fig3.show()

##### Based on the above histogram, the huge majority of the inventory is from the last 20 years. There are a decent amount of unknown model years, but there are not many cars from before 2000 to choose from.

#### Scatterplots

In [ ]:
df[df['age'].isna()]

In [ ]:
#making a scatter plot to see if the mileage of the car has an effect on the price
fig4 = px.scatter(df[df['age'].notna()],
                  x='odometer',
                  y='price',
                  color='age',
                  title='<b> Mileage vs. Price <b>',
                  template='plotly_dark')

fig4.update_xaxes(range=[0, 200000])
fig4.update_yaxes(range=[0, 75000])
fig4.update_layout(yaxis_title='Price (USD)',xaxis_title='Mileage',height=900)
fig4.show()

##### You can see from the scatter plot above that the higher the mileage of the car, the slightly lower price the car will be. There's not too many cars for under $10k that have less than 40k miles. As you get towards the higher mileages in the scatter plot, the cars are typically very old, but there are a fair amount of old cars with less than 100k miles.

In [ ]:
#making a scatter plot to see if days listed on the inventory has any effect on the price
fig5 = px.scatter(df,
                  x='days_listed',
                  y='price',
                  color='condition',
                  labels={'days_listed':'Days Listed','price':'Price'},
                  hover_data=['price','days_listed','condition','odometer'],
                  title='<b> Days Listed vs. Price <b>',
                  template='plotly_dark')

fig5.update_xaxes(range=[0, 200])
fig5.update_yaxes(range=[0, 60000])
fig5.update_layout(height=900)
fig5.show()

##### You can see above that most of the cars in the inventory haven't been listed for more than 100 days. The price ranges between cars 100 days and under do not vary that much.

##### But below, if you take out the majority of the inventory and limit to 100-200 days on the site, you can see that those cars that have been sitting unbought for a while are typically much cheaper. There must be problems with them

In [ ]:
fig6 = px.scatter(df,
                  x='days_listed',
                  y='price',
                  color='age',
                  title='<b> Days Listed between 100-200 Days vs. Price <b>',
                  template='plotly_dark')

fig6.update_xaxes(range=[100, 200])
fig6.update_yaxes(range=[0, 75000])
fig6.update_layout(yaxis_title='Price (USD)',xaxis_title='Days Listed',height=900)
fig6.show()

### Conclusion

I reviewed the vehicles_us.csv data, created functions to clean the data and created some visualizaitons to get a feel for the data I was working with

I see that there is a large variety of cars ranging from new to old, used to like new, cheap to expensive, and all types of different brands.

In my application, I allowed the user to filter the inventory by the make and the condition they would like to shop for as well as letting them filter by the year of the model and select new listings only.

I also gave the users some visualizaitons like the types of cars available by manufacturer and allowed them to compare between two different brands.

I provided a scatter plot of Mileage vs. Price which aimed to show how the more miles on a car, the cheaper it is likely to be in the market.

